In [ ]:
%pylab inline
import torch
import sys, os
import pystk
import ray
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print('device = ', device)
ray.init(logging_level=50)

Populating the interactive namespace from numpy and matplotlib


/home/philkr/.local/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  warnings.warn(


device =  cuda


{'node_ip_address': '128.83.143.189',
 'raylet_ip_address': '128.83.143.189',
 'redis_address': '128.83.143.189:6379',
 'object_store_address': '/tmp/ray/session_2021-09-10_14-35-52_461220_544108/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-09-10_14-35-52_461220_544108/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-09-10_14-35-52_461220_544108',
 'metrics_export_port': 61287,
 'node_id': '3d607b234774bf9382a32107f579c8ab270c6fc1c381ea41a2caee9c'}

In [ ]:
@ray.remote
class Rollout:
    def __init__(self, screen_width, screen_height, hd=True, track='lighthouse', render=True, frame_skip=1):
        # Init supertuxkart
        if not render:
            config = pystk.GraphicsConfig.none()
        elif hd:
            config = pystk.GraphicsConfig.hd()
        else:
            config = pystk.GraphicsConfig.ld()
        config.screen_width = screen_width
        config.screen_height = screen_height
        pystk.init(config)

        self.frame_skip = frame_skip
        self.render = render
        race_config = pystk.RaceConfig(track=track)
        self.race = pystk.Race(race_config)
        self.race.start()

    def __call__(self, agent, n_steps=200):
        torch.set_num_threads(1)
        self.race.restart()
        self.race.step()
        data = []
        track_info = pystk.Track()
        track_info.update()

        for i in range(n_steps // self.frame_skip):
            world_info = pystk.WorldState()
            world_info.update()

            # Gather world information
            kart_info = world_info.players[0].kart

            agent_data = {'track_info': track_info, 'kart_info': kart_info}
            if self.render:
                agent_data['image'] = np.array(self.race.render_data[0].image)

            # Act
            action = agent(**agent_data)
            agent_data['action'] = action

            # Take a step in the simulation
            for it in range(self.frame_skip):
                self.race.step(action)

            # Save all the relevant data
            data.append(agent_data)
        return data

def show_video(frames, fps=30):
    import imageio
    from IPython.display import Video, display

    imageio.mimwrite('/tmp/test.mp4', frames, fps=fps, bitrate=1000000)
    display(Video('/tmp/test.mp4', width=800, height=600, embed=True))

viz_rollout = Rollout.remote(400, 300)
def show_agent(agent, n_steps=600):
    data = ray.get(viz_rollout.__call__.remote(agent, n_steps=n_steps))
    show_video([d['image'] for d in data])

rollouts = [Rollout.remote(50, 50, hd=False, render=False, frame_skip=5) for i in range(10)]
def rollout_many(many_agents, **kwargs):
    ray_data = []
    for i, agent in enumerate(many_agents):
         ray_data.append( rollouts[i % len(rollouts)].__call__.remote(agent, **kwargs) )
    return ray.get(ray_data)

def dummy_agent(**kwargs):
    action = pystk.Action()
    action.acceleration = 1
    return action

In [ ]:
def three_points_on_track(distance, track):
    distance = np.clip(distance, track.path_distance[0,0], track.path_distance[-1,1]).astype(np.float32)
    valid_node = (track.path_distance[..., 0] <= distance) & (distance <= track.path_distance[..., 1])
    valid_node_idx, = np.where(valid_node)
    node_idx = valid_node_idx[0] # np.random.choice(valid_node_idx)
    d = track.path_distance[node_idx].astype(np.float32)
    x = track.path_nodes[node_idx][:,[0,2]].astype(np.float32) # Ignore the y coordinate
    w, = track.path_width[node_idx].astype(np.float32)

    t = (distance - d[0]) / (d[1] - d[0])
    mid = x[1] * t + x[0] * (1 - t)
    x10 = (x[1] - x[0]) / np.linalg.norm(x[1]-x[0])
    x10_ortho = np.array([-x10[1],x10[0]], dtype=float32)
    return mid - w / 2 * x10_ortho, mid, mid + w / 2 * x10_ortho


def state_features(track_info, kart_info, absolute=False, **kwargs):
    f = np.concatenate([three_points_on_track(kart_info.distance_down_track + d, track_info) for d in [0,5,10,15,20]])
    if absolute:
        return f
    p = np.array(kart_info.location)[[0,2]].astype(np.float32)
    t = np.array(kart_info.front)[[0,2]].astype(np.float32)
    f = f - p[None]
    d = (p-t) / np.linalg.norm(p-t)
    d_o = np.array([-d[1], d[0]], dtype=float32)
    return np.stack([f.dot(d), f.dot(d_o)], axis=1)

# Let's load a fancy auto-pilot. You'll write one yourself in your homework.
from _auto_pilot import auto_pilot
data, = rollout_many([auto_pilot], n_steps=400)

figure()
f = state_features(**data[50])
plot(f[:,1].flat, f[:,0].flat, '*')
axis('equal')
gca().invert_yaxis()

figure()
for d in data:
    f = state_features(**d, absolute=True)
    plot(f[:,1].flat, f[:,0].flat, '*')
axis('equal')
gca().invert_yaxis()

(pid=544552) [fatal  ] FileManager: Failed to locate PySuperTuxKartData. Got '/home/philkr/.local/lib/python3.8/site-packages/pystk_data', but data not found!
(pid=544552) 
(pid=544551) [fatal  ] FileManager: Failed to locate PySuperTuxKartData. Got '/home/philkr/.local/lib/python3.8/site-packages/pystk_data', but data not found!
(pid=544551) 
(pid=544556) [fatal  ] FileManager: Failed to locate PySuperTuxKartData. Got '/home/philkr/.local/lib/python3.8/site-packages/pystk_data', but data not found!
(pid=544556) 
(pid=544553) [fatal  ] FileManager: Failed to locate PySuperTuxKartData. Got '/home/philkr/.local/lib/python3.8/site-packages/pystk_data', but data not found!
(pid=544553) 
(pid=544545) [fatal  ] FileManager: Failed to locate PySuperTuxKartData. Got '/home/philkr/.local/lib/python3.8/site-packages/pystk_data', but data not found!
(pid=544545) 
(pid=544555) [fatal  ] FileManager: Failed to locate PySuperTuxKartData. Got '/home/philkr/.local/lib/python3.8/site-packages/pystk_dat

(pid=544545) 2021-09-10 14:35:55,381	ERROR worker.py:421 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::Rollout.__init__() (pid=544545, ip=128.83.143.189)
(pid=544545)   File "python/ray/_raylet.pyx", line 534, in ray._raylet.execute_task
(pid=544545)   File "python/ray/_raylet.pyx", line 484, in ray._raylet.execute_task.function_executor
(pid=544545)   File "/home/philkr/.local/lib/python3.8/site-packages/ray/_private/function_manager.py", line 563, in actor_method_executor
(pid=544545)     return method(__ray_actor, *args, **kwargs)
(pid=544545)   File "<ipython-input-2-9634ba3e13e3>", line 13, in __init__
(pid=544545) RuntimeError: Log::fatal
(pid=544552) 2021-09-10 14:35:55,403	ERROR worker.py:421 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::Rollout.__init__() (pid=544552, ip=128.83.143.189)
(pid=544552)   File "python/ray/_raylet.pyx", line 534, in ray._raylet.

RayActorError: The actor died because of an error raised in its creation task, [36mray::Rollout.__init__()[39m (pid=544549, ip=128.83.143.189)
  File "python/ray/_raylet.pyx", line 534, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 484, in ray._raylet.execute_task.function_executor
  File "/home/philkr/.local/lib/python3.8/site-packages/ray/_private/function_manager.py", line 563, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "<ipython-input-2-9634ba3e13e3>", line 13, in __init__
RuntimeError: Log::fatal

In [ ]:
# START OF GRADIENT FREE OPTIMIZATION


def new_action_net():
    return torch.nn.Linear(2*5*3, 1, bias=False) # linear classifier with no bias

class Actor:
    def __init__(self, action_net):
        self.action_net = action_net.cpu().eval()

    def __call__(self, track_info, kart_info, **kwargs):
        f = state_features(track_info, kart_info)
        output = self.action_net(torch.as_tensor(f).view(1,-1))[0] # output of network

        action = pystk.Action()
        action.acceleration = 1 # always accelerating
        action.steer = output[0] # steering is according to output of our network, no drifting or any other actions
        return action

In [ ]:
action_net = new_action_net() # create instance of agent
show_agent(Actor(action_net)) # show the agent

In [ ]:
many_action_nets = [new_action_net() for i in range(10)] # create action nets

data = rollout_many([Actor(action_net) for action_net in many_action_nets], n_steps=600) # rollout actor for each

good_initialization = many_action_nets[ np.argmax([d[-1]['kart_info'].overall_distance for d in data]) ] # pick the good initialization as the arg max of the action nets

In [ ]:
show_agent(Actor(good_initialization)) # shows good initialization agent

In [ ]:
# iteratively randomly searching for better parameters of this policy

import copy

n_epochs = 20
n_step = 20

action_net = copy.deepcopy(good_initialization) # copy of initialization
best_action_net = copy.deepcopy(good_initialization) # copy of best action net, currently just the initial one
best_dist = 0

for epoch in range(n_epochs):
    eps = 1e-2

    w = 1*action_net.weight.data

    networks = []
    ray_data = []
    for i in range(n_step): # generate a bunch of networks with slight modifications
        dp = torch.randn(w.shape) * eps # add random noise

        # Try positive
        action_net_dp = copy.deepcopy(action_net) # copy the network
        action_net_dp.weight.data[:] += dp # update the weights

        networks.append(action_net_dp)
    data = rollout_many([Actor(network) for network in networks], n_steps=600) # evaluate all these networks for 600 time steps, compute distances traveled down track
    distances = [d[-1]['kart_info'].overall_distance for d in data]
    print(np.max(distances))
    action_net = networks[np.argmax(distances)] # take the one with the best distance
    if np.max(distances) > best_dist: # if actually better than previous best, save as new best action network
        best_dist = np.max(distances)
        best_action_net = action_net


In [ ]:
show_agent(Actor(action_net))

(pid=544547) [fatal  ] FileManager: Failed to locate PySuperTuxKartData. Got '/home/philkr/.local/lib/python3.8/site-packages/pystk_data', but data not found!
(pid=544547) 
(pid=544554) [fatal  ] FileManager: Failed to locate PySuperTuxKartData. Got '/home/philkr/.local/lib/python3.8/site-packages/pystk_data', but data not found!
(pid=544554) 
(pid=544550) [fatal  ] FileManager: Failed to locate PySuperTuxKartData. Got '/home/philkr/.local/lib/python3.8/site-packages/pystk_data', but data not found!
(pid=544550) 
(pid=544549) [fatal  ] FileManager: Failed to locate PySuperTuxKartData. Got '/home/philkr/.local/lib/python3.8/site-packages/pystk_data', but data not found!
(pid=544549) 


NameError: name 'Actor' is not defined